In [ ]:
import sys

import torch
import jacinle
import json
import pickle
import numpy as np

import jactorch

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from typing import Optional, Union, List, Dict
from jacinle.config.environ_v2 import configs, set_configs
from left.domain import create_domain_from_parsing
from left.models.model import LeftModel
import re

import torch
import torch.nn as nn


import jacinle
from jacinle.utils.container import GView
from jacinle.config.environ_v2 import configs, set_configs, def_configs
from concepts.benchmark.clevr.clevr_constants import g_attribute_concepts, g_relational_concepts
from left.models.model import LeftModel
import json
import os

from experiments.desc_clevr_nesycoco import *



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
### load glove file
filename = "./word_embeds/glove/glove.6B.300d.txt"
glove = {}
with open(filename) as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        glove[word] = vector

def get_synonym(word):
    results = [glove[x] for x in word.split("_") if x in glove]
    np_results = np.array(results)
    ##sum
    if len(np_results) == 0:
        return None
    return np_results.sum(axis=0).reshape(1, -1)

def get_cosine_similarity(token_1, token_2):
    return cosine_similarity(get_synonym(token_1), get_synonym(token_2))[0][0]

token_1 = "congruent_shape"
token_2 = "same_shape"
get_cosine_similarity(token_1, token_2)

In [ ]:
import jacinle.io as io
from os import path as osp

og_parses = {}
og_parses.update(io.load("data/clevr/val/questions-ncprogram-gt.pkl"))
syn_parses = {}
syn_parses.update(io.load("data/clevr/val-syn-medium/questions-ncprogram-gt.pkl"))
data_dir = "data/clevr/val/"

data_questions_json = osp.join(data_dir, 'questions.json')
data_scenes_json = osp.join(data_dir, 'scenes.json')
data_image_root = osp.join(data_dir, 'images')
data_vocab_json = osp.join(data_dir, 'vocab.json')
data_output_vocab_json = osp.join(data_dir, 'output-vocab.json')
from left.domain import create_domain_from_parsing
### limit the number of parses
# all_parses = dict(list(all_parses.items())[:len(all_parses)//2])

og_domain = create_domain_from_parsing(og_parses)
syn_domain = create_domain_from_parsing(syn_parses)


from concepts.benchmark.clevr.dataset import make_dataset
dataset = make_dataset(
    data_scenes_json,
    data_questions_json,
    data_image_root,
    vocab_json=data_vocab_json,
    output_vocab_json=data_output_vocab_json,
)
# train_dataset, validation_dataset = dataset.split_trainval(0.95)
validation_dataset = dataset

print("Finsihed loading data")

In [ ]:
batch_size = 1
num_workers = 96
validation_dataloader = validation_dataset.make_dataloader(batch_size, shuffle=True, drop_last=False, nr_workers=num_workers)

from jacinle.config.g import g
g.concept_mapping = None


In [ ]:
len(dataset)

In [ ]:
from jactorch.io import state_dict, load_state_dict
from left.data.referit3d.vocabulary import Vocabulary
from jactorch.train import TrainerEnv
from jactorch.optim import AdamW
from jactorch.cuda.copy import async_copy_to

vocab = Vocabulary()
og_model = make_model(None, og_domain, og_parses, validation_dataset.output_vocab if hasattr(validation_dataset, 'output_vocab') else validation_dataset.unwrapped.output_vocab, custom_transfer=None)
og_model.cuda()
trainable_parameters = filter(lambda x: x.requires_grad, og_model.parameters())
optimizer = AdamW(trainable_parameters, 0.001 , weight_decay=1e-2)
og_trainer = TrainerEnv(og_model, optimizer)

syn_model = make_model(None, syn_domain, syn_parses, validation_dataset.output_vocab if hasattr(validation_dataset, 'output_vocab') else validation_dataset.unwrapped.output_vocab, custom_transfer=None)
syn_model.cuda()
trainable_parameters = filter(lambda x: x.requires_grad, syn_model.parameters())
optimizer = AdamW(trainable_parameters, 0.001 , weight_decay=1e-2)
syn_trainer = TrainerEnv(syn_model, optimizer)


filename = "[path to the model]"

og_trainer.load_checkpoint(filename)
og_model.eval()

syn_trainer.load_checkpoint(filename)
syn_model.eval()


In [ ]:
iterator = iter(validation_dataloader)
data_all = [next(iterator) for _ in range(200)]


In [ ]:
# Required imports
from generate_clevr_syn import validation_convert_dict
import numpy as np
from scipy.stats import kendalltau, pearsonr
from jacinle.utils.tqdm import tqdm_pbar
import matplotlib.pyplot as plt

# Initialize the group meters (if needed by your environment)
group_meters = jacinle.GroupMeters()
group_meters.reset()

# List to hold the concepts and their corresponding synonyms
replaced_concepts = []
for concept in validation_convert_dict:
    replaced_concepts.extend(validation_convert_dict[concept])

# Dictionaries to hold Kendall's Tau and p-values for each synonym pair
all_pearsons = {x: [] for x in replaced_concepts}
all_p_values = {x: [] for x in replaced_concepts}

# # Loop through the validation dataloader
# with tqdm_pbar(total=len(validation_dataloader)) as pbar:
#     for (index, feed_dict) in enumerate(validation_dataloader):

with tqdm_pbar(total=len(data_all)) as pbar:
    for (index, feed_dict) in enumerate(data_all):

        # Perform the necessary data transfer to GPU (or the appropriate device)
        feed_dict = async_copy_to(feed_dict, 0)

        # Evaluate the original and synonym models
        og_output_dict, og_extra_info = og_trainer.evaluate(feed_dict)
        syn_output_dict, syn_extra_info = syn_trainer.evaluate(feed_dict)

        # Loop through the execution traces to compare tensors
        for i in range(len(og_output_dict["execution_traces"][0])):
            if len(str(og_output_dict["execution_traces"][0][i][0])) > 6:
                og_fn = str(og_output_dict["execution_traces"][0][i][0])
                tensor_values = np.array(og_output_dict["execution_traces"][0][i][1].tensor.detach().cpu().numpy())
                syn_fn = str(syn_output_dict["execution_traces"][0][i][0])
                syn_predicate = syn_fn.split("(")[0].replace("_Object", "")
                og_predicate = og_fn.split("(")[0].replace("_Object", "")
                
                # Check for predicate mismatch and calculate Kendall's Tau
                if syn_predicate != og_predicate and syn_predicate in all_pearsons:
                    syn_tensor_values = np.array(syn_output_dict["execution_traces"][0][i][1].tensor.detach().cpu().numpy())
                    # if np.isnan(syn_tensor_values).any() or np.isnan(tensor_values).any() or np.inf in syn_tensor_values or np.inf in tensor_values:
                    #     continue
                    # syn_tensor_values = (syn_tensor_values - syn_tensor_values.min()) / (syn_tensor_values.max() - syn_tensor_values.min())            
                    corr, p_value = pearsonr(syn_tensor_values, tensor_values)

                    if np.isnan(corr) or np.isnan(p_value):
                        continue
                    
                    # Append the results to the corresponding lists
                    all_pearsons[syn_predicate].append(corr)
                    all_p_values[syn_predicate].append(p_value)
        
        pbar.update(1)
        
        # Break the loop if the index reaches 4000 (as per your original code)
        if index == 10000:
            break


In [ ]:
# Lists to store data for plotting
cosine_similarities = []
pearson_values = []
syn_pairs = []

# Compute and print the final results
for key, syns in validation_convert_dict.items():
    for syn in syns:
        cosine_similarity_score = get_cosine_similarity(key, syn)
        mean_tau = np.mean(all_pearsons[syn])
        mean_p_value = np.mean(all_p_values[syn])
        count = len(all_pearsons[syn])
        
        print(key, syn, "get_cosine_similarity", cosine_similarity_score, "pearson-value", mean_tau, "p-value", mean_p_value, count)
        
        # Store the values for plotting
        cosine_similarities.append(cosine_similarity_score)
        pearson_values.append(mean_tau)
        syn_pairs.append(f"{key}-{syn}")

# Plot the scatter plot with synonyms as labels
plt.figure(figsize=(12, 8))

# Plot each point with its corresponding synonym token pair
for i in range(len(cosine_similarities)):
    plt.scatter(cosine_similarities[i], pearson_values[i], color='blue')
    plt.text(cosine_similarities[i] + 0.01, pearson_values[i] + 0.01, syn_pairs[i], fontsize=9)

# Calculate Pearson correlation coefficient for the plot title
pearson_corr, cor_p_value = pearsonr(cosine_similarities, pearson_values)
print("Pearson Correlation:", pearson_corr, cor_p_value)
### x and y should have same scale
plt.xlim(0, 1)
plt.ylim(-0.2, 1.1)

## make the font bigger


plt.title(f"Scatter Plot of Cosine Similarity vs Tau's Value\nPearson Correlation: {pearson_corr:.2f}")
plt.xlabel("Cosine Similarity")
plt.ylabel("Tau's Value")
plt.grid(True)
plt.show()


In [ ]:
iterator = iter(validation_dataloader)
data_all = [next(iterator) for _ in range(200)]

In [ ]:
%matplotlib inline
import numpy as np
from scipy.stats import kendalltau


import sys

## Open a real image from address draw bouding boxes for objects
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import json
# plt.close()
from copy import deepcopy
from reascan import get_image
feed_dict = data_all[wrong_indices[21]]
feed_dict = async_copy_to(feed_dict, 0)

print(feed_dict)
q = feed_dict["question_raw"][0]

og_output_dict, og_extra_info = og_trainer.evaluate(feed_dict)
syn_output_dict, syn_extra_info = syn_trainer.evaluate(feed_dict)
all_taus = {x:[] for x in replaced_concepts}
all_p_values = {x:[] for x in replaced_concepts}
image = Image.open(osp.join(data_image_root, feed_dict["image_filename"][0])).convert('RGB')
print(feed_dict["question_raw"])
for i in range(len(og_output_dict["execution_traces"][0])):
    
    if len(str(og_output_dict["execution_traces"][0][i][0])) > 6:
        tensor_values = np.array(og_output_dict["execution_traces"][0][i][1].tensor.detach().cpu().numpy())
        fig, ax = plt.subplots()
        ax.imshow(image)
        og_fn = str(og_output_dict["execution_traces"][0][i][0])
        for obj_index,obj in enumerate(feed_dict["objects_raw"][0]):
            # Create a Rectangle patch
            rect = patches.Rectangle((obj[0], obj[1]), obj[2] - obj[0], obj[3] - obj[1], linewidth=1, edgecolor='r', facecolor="none")
            ax.add_patch(rect)
            try:
                text = f"{obj_index} {round(float(tensor_values[obj_index]),4)} "
            except:
                print("##"*100)
                text = f"{obj_index} {tensor_values} "
            ax.text(obj[0], obj[1], text, color='white', fontsize=10, bbox=dict(facecolor='red', alpha=0.5))
        plt.title(str(og_output_dict["execution_traces"][0][i][0]))
        plt.show()
        print(tensor_values)
        syn_fn = str(syn_output_dict["execution_traces"][0][i][0])
        if og_fn != syn_fn and syn_fn.count("(") == 1:
            syn_tensor_values = np.array(syn_output_dict["execution_traces"][0][i][1].tensor.detach().cpu().numpy())
            
            fig, ax = plt.subplots()
            ax.imshow(image)
            for obj_index,obj in enumerate(feed_dict["objects_raw"][0]):
                rect = patches.Rectangle((obj[0], obj[1]), obj[2] - obj[0], obj[3] - obj[1], linewidth=1, edgecolor='r', facecolor="none")
                ax.add_patch(rect)
                try:
                    text = f"{obj_index} {round(float(syn_tensor_values[obj_index]),4)} "
                except:
                    print("##"*100)
                    text = f"{obj_index} {syn_tensor_values} "
                ax.text(obj[0], obj[1], text, color='white', fontsize=10, bbox=dict(facecolor='red', alpha=0.5))
            plt.title("###########" + str(syn_output_dict["execution_traces"][0][i][0]))
            plt.show()
            print(syn_tensor_values)
            tau, p_value = kendalltau(syn_tensor_values, tensor_values)
            predicate = syn_fn.split("(")[0]
            all_taus.append(tau)
            all_p_values.append(p_value)
    

print(all_taus)
print(all_p_values)